In [1]:
import numpy as np
import networkx as nx
import hypernetx as hnx
import hypernetx.algorithms.hypergraph_modularity as hmod
import pandas as pd
from utils import *
from collections import defaultdict
import community as community_louvain

## Processing data

In [2]:
H, truth, partitions = get_dblp_data('dblp.pkl')
edge_list, triangle_list = hgraph_to_sc(H)

In [3]:
edge_dist = np.zeros((2, 6), dtype=int)
num_edges = len(H.edges)

edge_dist[0,:] = [2, 3, 4, 5, 6, 7]

for edge in H.edges:
    node_list = [int(node) for node in edge.split("_")]
    edge_size = len(node_list)
    edge_dist[1,edge_size-2] += 1

print("Total number of edges:", np.sum(edge_dist[1,:]))
print("Edge distribution")
print(edge_dist)

Total number of edges: 1372
Edge distribution
[[  2   3   4   5   6   7]
 [739 447 149  30   6   1]]


In [4]:
HG = hmod.precompute_attributes(H)

In [5]:
K = hmod.kumar(HG)
K_strict = hmod.last_step(HG, K, hmod.strict)
K_maj =    hmod.last_step(HG, K, hmod.majority)
K_linear = hmod.last_step(HG, K, hmod.linear)

In [6]:
hgraph_mod_strict = hmod.modularity(HG, K_strict, hmod.strict)
hgraph_mod_maj =    hmod.modularity(HG, K_maj, hmod.majority)
hgraph_mod_linear = hmod.modularity(HG, K_linear, hmod.linear)
hgraph_res = np.array([hgraph_mod_strict, hgraph_mod_maj, hgraph_mod_linear])

## Wu et al.

In [7]:
edge_list = pd.DataFrame (edge_list, columns=['node_1', 'node_2'])
triangle_list = pd.DataFrame (triangle_list, columns=['node_1', 'node_2', 'node_3'])

G = nx.Graph()

unique_nodes = set()
for i in range(len(edge_list)):
    u = edge_list.iloc[i][0]
    v = edge_list.iloc[i][1]
    unique_nodes = unique_nodes | {u, v}
unique_nodes = list(unique_nodes)
unique_nodes.sort()
G.add_nodes_from(unique_nodes)

for i in range(len(edge_list)):
    u = edge_list.iloc[i][0]
    v = edge_list.iloc[i][1]
    G.add_edge(u,v)

In [8]:
B1 = generate_B1ca(G)
B2 = generate_B2ca_cc(G,triangle_list)

V = np.vstack((np.identity(len(G.edges)),-np.identity(len(G.edges))))

A_l_hat = generate_A_l_hat(B1,V)
A_u_hat = generate_A_u_hat(B2,V)
A_s_hat = generate_A_s_hat(B1,B2)
A_rw_hat = generate_A_rw_hat(A_s_hat, A_l_hat,A_u_hat)

#check the condition for Therorem 2:
if np.sum(A_rw_hat) < np.amax(np.sum(A_rw_hat,axis = 1)):
    print("assumption violation")

np.fill_diagonal(A_rw_hat, np.zeros(A_rw_hat.shape[1]))
np.zeros(A_rw_hat.shape[1])
G_dual = nx.from_numpy_matrix(A_rw_hat)
louvain_SC = community_louvain.best_partition(G_dual)

In [9]:
edge_class_SC = list(louvain_SC.values())[0:len(G.edges)]
node_class_SC = edge_comm_to_node_comm(G, edge_class_SC).items()
res_SC = defaultdict(list)
for key, val in sorted(node_class_SC):
    res_SC[val].append(key)

In [11]:
K_SC = [set(res_SC[i]) for i in res_SC]  

In [12]:
sc_mod_strict = hmod.modularity(HG, K_SC, hmod.strict)
sc_mod_maj = hmod.modularity(HG, K_SC, hmod.majority)
sc_mod_linear = hmod.modularity(HG, K_SC, hmod.linear)
sc_res = np.array([sc_mod_strict, sc_mod_maj, sc_mod_linear])

## Louvain method

In [13]:
res_graph = defaultdict(list)
for key, val in sorted(community_louvain.best_partition(G).items()):
    res_graph[val].append(key)
louv_part = [set(res_graph[i]) for i in res_graph]  

louv_mod_strict = hmod.modularity(HG, louv_part, hmod.strict)
louv_mod_maj = hmod.modularity(HG, louv_part, hmod.majority)
louv_mod_linear = hmod.modularity(HG, louv_part, hmod.linear)
louv_res = np.array([louv_mod_strict, louv_mod_maj, louv_mod_linear])

In [14]:
results = np.vstack([hgraph_res, sc_res, louv_res])
results = pd.DataFrame(results)
results.columns = ['strict', 'majority', 'linear']
results.index = ['Kaminski et al.', 'Wu et al.', 'Louvain']
results

,strict,majority,linear
Kaminski et al.,0.862536,0.902157,0.885742
Wu et al.,0.461235,0.496176,0.485762
Louvain,0.483609,0.486180,0.486059
